In [ ]:
import pandas as pd
import numpy as np
import seaborn as snb
import warnings
import matplotlib.pyplot as plt
import re
import string
from gensim.models import KeyedVectors, Word2Vec
from collections import Counter
from textblob import TextBlob
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords
from fuzzywuzzy import fuzz
from collections import Counter
import tensorflow as tf
from sklearn.model_selection import GroupKFold
import tensorflow_hub as hub
import random as rn
from scipy.stats import spearmanr
from transformers import XLNetConfig, TFXLNetModel, XLNetTokenizer, TFXLNetMainLayer
import tensorflow_hub as hub
from scipy.stats import spearmanr
import time
import os
warnings.filterwarnings('ignore')

In [59]:
class XLNet(TFXLNetModel):
    def __init__(self, config, *inputs, **kwrgs):
        super(XLNet, self).__init__(config, *inputs, **kwrgs)
        self.transformer = TFXLNetMainLayer(config, name = 'transformer')
        for i in range(1, 3):
            self.transformer.layer[-i].trainable = False
    def call(self, inputs, **kwrgs):
        outputs = self.transformer(inputs)
        hidden_states = outputs[1]
        h12 = hidden_states[-1][:, 0, :]
        h11 = hidden_states[-2][:, 0, :]
        h10 = hidden_states[-3][:, 0, :]
        h9 = hidden_states[-4][:, 0, :]
        concat = tf.keras.layers.Concatenate(axis = -1)([h9, h10, h11, h12])
        return concat
def transformer_model(name):
    id_1 = tf.keras.Input(shape = (512), dtype = tf.int32)
    id_2 = tf.keras.Input(shape = (512), dtype = tf.int32)

    type_id_1 = tf.keras.Input(shape = (512), dtype = tf.int32)
    type_id_2 = tf.keras.Input(shape = (512), dtype = tf.int32)

    a1 = tf.keras.Input(shape = (512), dtype = tf.int32)
    a2 = tf.keras.Input(shape = (512), dtype = tf.int32)

    config = XLNetConfig.from_pretrained('xlnet/config-xlnet-base-cased', output_hidden_states = True)
    transformer = XLNet.from_pretrained('xlnet/model-xlnet-base-cased', config = config)

  
    out_1 = transformer({'input_ids':id_1, 'attention_mask':a1, 'token_type_ids':type_id_1})
    out_2 = transformer({'input_ids':id_2, 'attention_mask':a2, 'token_type_ids':type_id_2})
  
    concat = tf.keras.layers.Concatenate(axis = -1)([out_1, out_2])
    drop = tf.keras.layers.Dropout(rate = 0.1)(concat)
    dense = tf.keras.layers.Dense(30, activation = 'sigmoid')(drop)
    return tf.keras.Model(inputs = [id_1, id_2, type_id_1, type_id_2, a1, a2], outputs = [dense])
def SpearmanCorrCoeff(A, B):
        overall_score = []
        for index in range(A.shape[1]):
            overall_score += [spearmanr(A[:, index], B[:, index]).correlation]
        return overall_score

In [ ]:
class Optimize:
    def __init__(self):
        self.clips = [[0, 1] for i in range(30)]
        self.ab_ = [(0, 0.15), (0.85, 1)]
        self.new_scores, self.scores = (None, None)
    def fit(self, labels, preds):
        self.scores = [SpearmanCorrCoeff(labels[:, i:i+1], preds[:, i:i+1]) for i in range(30)]
        for i in range(30):
            self.golden_section_search(labels[:, i:i+1], preds[:, i:i+1], i, 0)
            self.golden_section_search(labels[:, i:i+1], preds[:, i:i+1], i, 1)
        self.new_scores = [np.nan_to_num(SpearmanCorrCoeff(labels[:, i:i+1], np.clip(preds[:, i:i+1], self.clips[i][0], self.clips[i][1]))) for i in range(30)]
        for i in range(30):
            if self.scores[i] >= self.new_scores[i]:
                self.clips[i] = [0, 1]
    def golden_section_search(self, labels, preds, i, idx):
        (a, b) = self.ab_[idx]
        c = 0.618
        x1 = b - c*(b-a)
        x2 = (b-a)*c + a
        
        for epochs in range(20):
            self.clips[i][idx] = x1
            score_a = -self.score(labels, preds, i)
            self.clips[i][idx] = x2
            score_b = -self.score(labels, preds, i)
            if np.isnan(score_a):
                score_a = 0
            elif np.isnan(score_b):
                score_b = 0
            elif score_a <= score_b:
                b = x2
                x2 = x1
                x1 = b - c*(b-a)
            else:
                a = x1
                x1 = x2
                x2 = (b-a)*c + a
        
        self.clips[i][idx] = x1
        score_x1 = self.score(labels, preds, i)
        self.clips[i][idx] = x2
        score_x2 = self.score(labels, preds, i)
        if score_x1 > score_x2:
            self.clips[i][idx] = x1
        else:
            self.clips[i][idx] = x2
                    
            
    def score(self, labels, preds, i):
        return SpearmanCorrCoeff(labels, np.clip(preds, self.clips[i][0], self.clips[i][1]))
    def transform(self, preds):
        temp = preds.copy()
        for i in range(30):
            clipped = np.clip(preds[:, i], self.clips[i][0], self.clips[i][1])
            if np.unique(clipped).shape[0] > 1:
                temp[:, i][:] = clipped
        return temp

In [78]:
def get_inputs(df):
    '''This function is used to preprocess and model-ready inputs from text data.'''

    contractions = {"ain't": "is not","aren't": "are not","can't": "cannot",
    "can't've": "cannot have","'cause": "because","could've": "could have",
    "couldn't": "could not","couldn't've": "could not have","didn't": "did not",
    "doesn't": "does not","don't": "do not","hadn't": "had not","hadn't've": "had not have",
    "hasn't": "has not","haven't": "have not","he'd": "he would",
    "he'd've": "he would have","he'll": "he will","he'll've": "he he will have",
    "he's": "he is","how'd": "how did","how'd'y": "how do you","how'll": "how will",
    "how's": "how is","I'd": "I would","i'd've": "I would have","i'll": "I will",
    "i'll've": "I will have","i'm": "I am","i've": "I have","i'd": "i would",
    "i'd've": "i would have","i'll": "i will","i'll've": "i will have","i'm": "i am",
    "i've": "i have","isn't": "is not","its":"it is","it'd": "it would",
    "it'd've": "it would have","it'll": "it will","it'll've": "it will have","it's": 
    "it is","let's": "let us","ma'am": "madam","mayn't": "may not",
    "might've": "might have","mightn't": "might not","mightn't've": "might not have",
    "must've": "must have","mustn't": "must not","mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not",
    "oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
    "shan't've": "shall not have","she'd": "she would","she'd've": "she would have",
    "she'll": "she will","she'll've": "she will have","she's": "she is",
    "should've": "should have","shouldn't": "should not","shouldn't've": "should not have",
    "so've": "so have","so's": "so as","that'd": "that would",
    "that'd've": "that would have","that's": "that is","there'd": "there would",
    "there'd've": "there would have","there's": "there is","they'd": "they would",
    "they'd've": "they would have","they'll": "they will","they'll've": "they will have",
    "they're": "they are","they've": "they have","to've": "to have",
    "wasn't": "was not","we'd": "we would","we'd've": "we would have","we'll": "we will",
    "we'll've": "we will have","we're": "we are","we've": "we have",
    "weren't": "were not","what'll": "what will","what'll've": "what will have",
    "what're": "what are","what's": "what is","what've": "what have",
    "when's": "when is","when've": "when have","where'd": "where did","where's": "where is",
    "where've": "where have","who'll": "who will",
    "who'll've": "who will have","who's": "who is","who've": "who have","why's": "why is",
    "why've": "why have","will've": "will have","won't": "will not",
    "won't've": "will not have","would've": "would have","wouldn't": "would not",
    "wouldn't've": "would not have","y'all": "you all","y'all'd": "you all would",
    "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
    "you'd": "you would","you'd've": "you would have","you'll": "you will",
    "you'll've": "you will have","you're": "you are","you've": "you have"
    }

    rules = {
        "'t": " not","'cause": " because","'ve": " have","'t": " not","'s": " is","'d": " had"
    }

    punctuations = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", 
                    '$', '&', '/', '[', ']','>', '%', '=', '#', '*', '+', '\\', '•', '~', 
                    '@', '£', '·', '_', '{', '}', '©', '^','®', '`', '<', '→', '°', '€', '™', 
                    '›', '♥', '←', '×', '§', '″', '′', 'Â', '█','½', 'à', '…', '“', '★', '”', 
                    '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶','↑', '±', '¿', '▾', '═', 
                    '¦', '║', '―', '¥', '▓', '—', '‹', '─', '▒', '：', '¼','⊕', '▼', '▪', '†',
                    '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲','è', '¸', '¾', 
                    'Ã', '⋅', '‘', '∞', '∙', '）', '↓', '、', '│', '（', '»', '，', '♪','╩', 
                    '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 
                    'ï', 'Ø', '¹', '≤', '‡', '√'] + list(string.punctuation)
    token_replace = {
        'usepackage':'latex','orf19':'gene','documentclass':'latex','magento':'open-source e-commerce',
        'appium':'web-app','tikz':'programming language',
        'tikzpicture':'programming language','openvpn':'vpn','httpclient':'http client',
        'arraylist':'array list','jsonobject': 'json',
        'artifactid':'xml','hwnd':'os'
    }
    punctuations_in_embeddings = \
    {',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', 
                    '>', '%', '=', '#', '*', '+', '\\', '•', '~', '@', '£', '·', '{', '}', '©', '^', '®', 
                    '<', '→', '°', '€', '™', '›', '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
                    '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '═', 
                    '¦', '║', '―', '¥', '▓', '—', '‹', '─', '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', 
                    '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
                    '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔',
                    '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', '!', '"', '#', '$', '%', '&', "'", '(', ')', 
                    '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '{', 
                    '|', '}', '~'}
    
    '''Pipeline for text preprocessing. Check coverage computes what %age of tokens in the text data 
        are covered by Embeddings. preprocess_text is used to clean text.
    '''
    def improve_text(dframes):
        def preprocess_text(s):
            s = s.lower()
            '''Expanding contractions...'''
            for key, value in contractions.items():
                s = s.replace(key, f' {value} ')
            for key, value in rules.items():
                s = s.replace(key, f' {value} ')
            '''Fixing punctuations...'''
            for punct in punctuations:
                if punct in punctuations_in_embeddings:
                    s = s.replace(punct, f' {punct} ')
                else:
                    s = s.replace(punct, ' ')
            '''Replacing few tokens with its similar word/group of words...'''
            for key, value in token_replace.items():
                s = s.replace(key, value)
            '''Removing HTML tags'''
            s = re.sub('<.*?>', ' ', s)
            s = re.sub('\s+', ' ', s)
            return s
        print("Applying Preprocessing.....")
        for each_df in dframes:
            each_df['clean_title'] = each_df['question_title'].apply(preprocess_text)
            each_df['clean_question_body'] = each_df['question_body'].apply(preprocess_text)
            each_df['clean_answer'] = each_df['answer'].apply(preprocess_text)
        print("Preprocessing Finished")

    def tokenize_input(tokenizer, s1, s2, tags, data_name, max_length):
        if s2 is not None:
            x = tokenizer.encode_plus(s1, s2, pad_to_max_length=False)
            if len(x['input_ids']) > max_length:
                segment_1 = int(0.25*max_length)
                x['input_ids'] = x['input_ids'][:segment_1] + x['input_ids'][-(max_length-segment_1):]
                x['attention_mask'] = x['attention_mask'][:segment_1] + x['attention_mask'][-(max_length-segment_1):]
                x['token_type_ids'] = x['token_type_ids'][:segment_1] + x['token_type_ids'][-(max_length-segment_1):]
            else:
                diff = max_length - len(x['input_ids'])
                x['input_ids'] = [tokenizer.pad_token_id]*diff + x['input_ids']
                x['attention_mask'] = [0]*diff + x['attention_mask']
                x['token_type_ids'] = [tokenizer.pad_token_type_id]*diff + x['token_type_ids']

        else:
            x = tokenizer.encode_plus(s1)
            if len(x['input_ids']) > max_length:
                segment_1 = int(0.25*max_length)
                x['input_ids'] = x['input_ids'][:segment_1] + x['input_ids'][-(max_length-segment_1):]
                x['attention_mask'] = x['attention_mask'][:segment_1] + x['attention_mask'][-(max_length-segment_1):]
                x['token_type_ids'] = x['token_type_ids'][:segment_1] + x['token_type_ids'][-(max_length-segment_1):]
            else:
                diff = max_length - len(x['input_ids'])
                x['input_ids'] = [tokenizer.pad_token_id]*diff + x['input_ids']
                x['attention_mask'] = [0]*diff + x['attention_mask']
                x['token_type_ids'] = [tokenizer.pad_token_type_id]*diff + x['token_type_ids']

        data[data_name][tags[0]].append(x['input_ids']) 
        data[data_name][tags[1]].append(x['token_type_ids'])
        data[data_name][tags[2]].append(x['attention_mask']) 

    def is_eng(x):
        if ((x == 'english') | (x == 'eli')):
            return 1
        return 0
    def question_type(row):
        if ((row['is_eng'] == 1) & (row['a_count'] > 95)):
            return 1
        else:
            return 0
    def fetch_words(s, use_stopwords = False, ignore_stopwords = False):
        if ignore_stopwords == True:
            return {word for word in s.split() if word not in stop_words}
        elif use_stopwords:
            return {word for word in s.split() if word in stop_words}
        else:
            return set(s.split())
    def text_features(row, use_stopwords, is_token, return_max):
        t1, t2 = None, None
        if use_stopwords == True:
            t1 = fetch_words(row[0], use_stopwords = True)
            t2 = fetch_words(row[1], use_stopwords = True)
        elif is_token == True:
            t1 = fetch_words(row[0], ignore_stopwords = True)
            t2 = fetch_words(row[1], ignore_stopwords = True)
        else:
            t1 = fetch_words(row[0])
            t2 = fetch_words(row[1])
        if return_max:
            try:
                ans = len(t1.intersection(t2))/max(len(t1), len(t2))
            except:
                return 0
        else:
            try:
                ans = len(t1.intersection(t2))/min(len(t1), len(t2))
            except:
                return 0
        return ans

    def get_ratio(row, is_token = False):
        if is_token == True:
            t1 = len(set([word for word in row[0].split() if word not in stop_words]))
            t2 = len(set([word for word in row[1].split() if word not in stop_words]))
        t1 = len(set([word for word in row[0].split()]))
        t2 = len(set([word for word in row[1].split()]))
        try:
            return t1/t2
        except:
            return 0
    def get_fuzzy_ratio(row, ratio, partial_ratio, token_sort_ratio, token_set_ratio):
        if ratio:
            return fuzz.ratio(row[0], row[1])/100
        elif partial_ratio:
            return fuzz.partial_ratio(row[0], row[1])/100
        elif token_sort_ratio:
            return fuzz.token_sort_ratio(row[0], row[1])/100
        else:
            return fuzz.token_set_ratio(row[0], row[1])/100
        
    def get_polarity(s):
        return TextBlob(s).sentiment.polarity
    def get_token_ids(texts, max_length):
        tokens = []
        for text in texts:
            tmp_tokens = []
            if len(text.split()) > max_length:
                for each in (text.split()[:(max_length//2)] + text.split()[-(max_length//2):]):
                    tmp_tokens.append(vocab['token2id'].get(each, 0))
                tokens.append(tmp_tokens)
            else:
                for each in (text.split()[:max_length]):
                    tmp_tokens.append(vocab['token2id'].get(each, 0))
                tokens.append(tmp_tokens)
        return tf.keras.preprocessing.sequence.pad_sequences(tokens, padding="post", maxlen=max_length)

    feature_engg_cols = ['cwc_min_title_question_body',
       'cwc_max_title_question_body', 'csc_min_title_question_body',
       'csc_max_title_question_body', 'ctc_min_title_question_body',
       'ctc_max_title_question_body', 'cwc_min_title_answer',
       'cwc_max_title_answer', 'csc_min_title_answer', 'csc_max_title_answer',
       'ctc_min_title_answer', 'ctc_max_title_answer',
       'cwc_min_question_body_answer', 'cwc_max_question_body_answer',
       'csc_min_question_body_answer', 'csc_max_question_body_answer',
       'ctc_min_question_body_answer', 'ctc_max_question_body_answer',
       'word_title_question_body_ratio', 'token_title_question_body_ratio',
       'word_question_body_answer_ratio', 'token_question_body_answer_ratio',
       'word_title_answer_ratio', 'token_title_answer_ratio',
       'ratio_title_question_body', 'partial_ratio_title_question_body',
       'token_sort_ratio_title_question_body',
       'token_set_ratio_title_question_body', 'ratio_question_body_answer',
       'partial_ratio_question_body_answer',
       'token_sort_ratio_question_body_answer',
       'token_set_ratio_question_body_answer', 'ratio_title_answer',
       'partial_ratio_title_answer', 'token_sort_ratio_title_answer',
       'token_set_ratio_title_answer', 'question_title_polarity', 'question_body_polarity', 'answer_polarity']
    
    improve_text([df])
    
    df['domain'] = df.host.apply(lambda s:s.split('.')[0])
    df['is_eng'] = df.domain.apply(lambda x:is_eng(x))
    df['a_count'] = df.answer.apply(lambda x: len(x.split()))
    df['question_type_spelling_modified'] = df[['is_eng', 'a_count']].apply(question_type, axis = 1)
    
    df['question_title_polarity'] = df.clean_title.apply(get_polarity)
    df['question_body_polarity'] = df.clean_question_body.apply(get_polarity)
    df['answer_polarity'] = df.clean_answer.apply(get_polarity)
    df['question_title_count'] = df.clean_title.apply(lambda x: len(x.split()))
    df['question_body_count'] = df.clean_question_body.apply(lambda x: len(x.split()))
    df['answer_count'] = df.clean_answer.apply(lambda x: len(x.split()))
    
    booleans = [(False, False, False), (False, False, True), (True, False, False), (True, False, True), (False, True, False),
                (False, True, True)]
    for i in range(len(booleans)):
        df[feature_engg_cols[i]] = df[['clean_title', 'clean_question_body']].apply(
        func = text_features, axis = 1, args = booleans[i])
    
    for i in range(len(booleans)):
        df[feature_engg_cols[i+6]] = df[['clean_title', 'clean_answer']].apply(
        func = text_features, axis = 1, args = booleans[i])

    for i in range(len(booleans)):
        df[feature_engg_cols[i+12]] = df[['clean_question_body', 'clean_answer']].apply(
        func = text_features, axis = 1, args = booleans[i])


    df['word_title_question_body_ratio'] = df[['clean_title', 'clean_question_body']].apply(
        func = get_ratio, axis = 1, args = (False, ))
    df['token_title_question_body_ratio'] = df[['clean_title', 'clean_question_body']].apply(
        func = get_ratio, axis = 1, args = (True, ))
    df['word_question_body_answer_ratio'] = df[['clean_question_body', 'clean_answer']].apply(
        func = get_ratio, axis = 1, args = (False, ))
    df['token_question_body_answer_ratio'] = df[['clean_question_body', 'clean_answer']].apply(
        func = get_ratio, axis = 1, args = (True, ))
    df['word_title_answer_ratio'] = df[['clean_title', 'clean_answer']].apply(func = get_ratio, 
                                                    axis = 1, args = (False, ))
    df['token_title_answer_ratio'] = df[['clean_title', 'clean_answer']].apply(func = get_ratio, 
                                                    axis = 1, args = (True, ))

    df['ratio_title_question_body'] = \
    df[['clean_title', 'clean_question_body']].apply(func = get_fuzzy_ratio, 
                                                     axis = 1, args = (True, False, False, False))
    df['partial_ratio_title_question_body'] = \
    df[['clean_title', 'clean_question_body']].apply(func = get_fuzzy_ratio, 
                                                     axis = 1, args = (False, True, False, False))
    df['token_sort_ratio_title_question_body'] = \
    df[['clean_title', 'clean_question_body']].apply(func = get_fuzzy_ratio, 
                                                     axis = 1, args = (False, False, True, False))
    df['token_set_ratio_title_question_body'] = \
    df[['clean_title', 'clean_question_body']].apply(func = get_fuzzy_ratio, 
                                                     axis = 1, args = (False, False, False, True))
    df['ratio_question_body_answer'] = \
    df[['clean_question_body', 'clean_answer']].apply(func = get_fuzzy_ratio, 
                                                     axis = 1, args = (True, False, False, False ))
    df['partial_ratio_question_body_answer'] = \
    df[['clean_question_body', 'clean_answer']].apply(func = get_fuzzy_ratio, 
                                                     axis = 1, args = (False, True, False, False))
    df['token_sort_ratio_question_body_answer'] = \
    df[['clean_question_body', 'clean_answer']].apply(func = get_fuzzy_ratio, 
                                                     axis = 1, args = (False, False, True, False))
    df['token_set_ratio_question_body_answer'] = \
    df[['clean_question_body', 'clean_answer']].apply(func = get_fuzzy_ratio, 
                                                     axis = 1, args = (False, False, False, True))
    df['ratio_title_answer'] = \
    df[['clean_title', 'clean_answer']].apply(func = get_fuzzy_ratio, 
                                                     axis = 1, args = (True, False, False, False))
    df['partial_ratio_title_answer'] = \
    df[['clean_title', 'clean_answer']].apply(func = get_fuzzy_ratio, 
                                                     axis = 1, args = (False, True, False, False))
    df['token_sort_ratio_title_answer'] = \
    df[['clean_title', 'clean_answer']].apply(func = get_fuzzy_ratio, 
                                                     axis = 1, args = (False, False, True, False))
    df['token_set_ratio_title_answer'] = \
    df[['clean_title', 'clean_answer']].apply(func = get_fuzzy_ratio, 
                                                     axis = 1, args = (False, False, False, True))
    
    
    
    df['clean_t_q'] = df['clean_title'] + ' ' + df['clean_question_body']
    
    data = {}
    
    data['xlnet_t_a'] = {}
    data['xlnet_q_a'] = {}
    data['xlnet_t_q'] = {}
    data['xlnet_q'] = {}
    data['xlnet_a'] = {}

    tags = ['input_ids', 'token_type_ids', 'attention_masks']
    data['xlnet_t_a'][tags[0]], data['xlnet_t_a'][tags[1]],data['xlnet_t_a'][tags[2]] = \
    [], [], []
    data['xlnet_q_a'][tags[0]], data['xlnet_q_a'][tags[1]], data['xlnet_q_a'][tags[2]] = \
    [], [], []
    data['xlnet_t_q'][tags[0]], data['xlnet_t_q'][tags[1]], data['xlnet_t_q'][tags[2]] = \
    [], [], []
    data['xlnet_q'][tags[0]], data['xlnet_q'][tags[1]], data['xlnet_q'][tags[2]] = \
    [], [], []
    data['xlnet_a'][tags[0]], data['xlnet_a'][tags[1]], data['xlnet_a'][tags[2]] = \
    [], [], []
    
    
    
    for i in range(df.shape[0]):
        tokenize_input(xlnet_tokenizer, df.iloc[i].question_title, 
                       df.iloc[i].answer, tags, 'xlnet_t_a', 512)
        tokenize_input(xlnet_tokenizer, df.iloc[i].question_body, 
                       df.iloc[i].answer, tags, 'xlnet_q_a', 512)
        tokenize_input(xlnet_tokenizer, df.iloc[i].question_title, 
                       df.iloc[i].question_body, tags, 'xlnet_t_q', 512)
        tokenize_input(xlnet_tokenizer, df.iloc[i].question_body, None, tags, 'xlnet_q', 512)
        tokenize_input(xlnet_tokenizer, df.iloc[i].answer, None, tags, 'xlnet_a', 512)
    for key, _ in data.items():
        for k, _ in data[key].items():
            data[key][k] = np.array(data[key][k])

    MAX_LENGTH = 512
    data['question_title'] = get_token_ids(df['clean_t_q'], MAX_LENGTH)
    data['answer'] = get_token_ids(df['clean_answer'], MAX_LENGTH)
    
    fe = df[feature_engg_cols]
    
    df['combine_title_question'] = df['question_title'] + ' ' + df['question_body']

    data['question_title_use'] = []
    data['answer_use'] = []

    BATCH_SIZE = 4

    for i in range(0, df.shape[0], BATCH_SIZE):
        data['question_title_use'] += [encoder.signatures['question_encoder'](
            
          tf.constant(df['combine_title_question'].iloc[i:i+BATCH_SIZE].tolist())
        
        )['outputs'].numpy().astype(np.float16)]
        data['answer_use'] += [encoder.signatures['response_encoder'](
            
        input = tf.constant(df['answer'].iloc[i:i+BATCH_SIZE].tolist()), 
        context = tf.constant(df['answer'].iloc[i:i+BATCH_SIZE].tolist())
        
        )['outputs'].numpy().astype(np.float16)]

    data['question_title_use'] = np.vstack(data['question_title_use'])
    data['answer_use'] = np.vstack(data['answer_use'])

    
    return data, fe

In [79]:
def function1(df):
    '''Takes input and outputs prediction'''

    tags = ['input_ids', 'token_type_ids', 'attention_masks']
    target_cols = ['question_asker_intent_understanding', 'question_body_critical', 'question_conversational', 'question_expect_short_answer', 
               'question_fact_seeking', 'question_has_commonly_accepted_answer', 'question_interestingness_others', 'question_interestingness_self', 
               'question_multi_intent', 'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice', 'question_type_compare', 
               'question_type_consequence', 'question_type_definition', 'question_type_entity', 'question_type_instructions', 'question_type_procedure', 
               'question_type_reason_explanation', 'question_type_spelling', 'question_well_written', 'answer_helpful', 'answer_level_of_information', 
               'answer_plausible', 'answer_relevance', 'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 'answer_type_reason_explanation', 
               'answer_well_written']
    data, fe = get_inputs(df)
    c_lstm_inputs = (data['question_title'], data['answer'],
                   data['question_title_use'], data['answer_use'], fe)
    
    xlnet_inputs = (
                    data['xlnet_a'][tags[0]], data['xlnet_t_q'][tags[0]], 
        data['xlnet_a'][tags[1]], data['xlnet_t_q'][tags[1]],
                   data['xlnet_a'][tags[2]], data['xlnet_t_q'][tags[2]]  
                
                 )
    xlnet_predictions = xlnet_model.predict(xlnet_inputs)
    c_lstm_predictions = c_lstm_model.predict(c_lstm_inputs)
    predictions = 0.5*(xlnet_predictions + c_lstm_predictions)
    predictions[:, 19] = np.multiply(predictions[:, 19], df['question_type_spelling_modified'].values)
    predictions = opt.transform(predictions)
    output = {}
    for i in range(30):
        output[target_cols[i]] = predictions[:, i]
    return pd.DataFrame(output)


def function2(df):
    '''Takes input and outputs Spearman Correlation Coefficient as a metric'''
    
    tags = ['input_ids', 'token_type_ids', 'attention_masks']
    target_cols = ['question_asker_intent_understanding', 'question_body_critical', 'question_conversational', 'question_expect_short_answer', 
               'question_fact_seeking', 'question_has_commonly_accepted_answer', 'question_interestingness_others', 'question_interestingness_self', 
               'question_multi_intent', 'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice', 'question_type_compare', 
               'question_type_consequence', 'question_type_definition', 'question_type_entity', 'question_type_instructions', 'question_type_procedure', 
               'question_type_reason_explanation', 'question_type_spelling', 'question_well_written', 'answer_helpful', 'answer_level_of_information', 
               'answer_plausible', 'answer_relevance', 'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 'answer_type_reason_explanation', 
               'answer_well_written']
    data, fe = get_inputs(df)
    c_lstm_inputs = (data['question_title'], data['answer'],
                   data['question_title_use'], data['answer_use'], fe)
    
    xlnet_inputs = (
                    data['xlnet_a'][tags[0]], data['xlnet_t_q'][tags[0]], 
        data['xlnet_a'][tags[1]], data['xlnet_t_q'][tags[1]],
                   data['xlnet_a'][tags[2]], data['xlnet_t_q'][tags[2]]  
                
                 )
    true_labels = df[target_cols].values.astype(np.float32) 
    xlnet_predictions = xlnet_model.predict(xlnet_inputs)
    c_lstm_predictions = c_lstm_model.predict(c_lstm_inputs)
    predictions = 0.5*(xlnet_predictions + c_lstm_predictions)
    predictions[:, 19] = np.multiply(1, df['question_type_spelling_modified'].values)
    scores = SpearmanCorrCoeff(predictions, true_labels)
    print(f"Spearman Corr Coeff (Before Optimizer): {np.round(np.mean(scores), 3)}")
    predictions = opt.transform(predictions)
    scores = SpearmanCorrCoeff(predictions, true_labels)
    print(f"Spearman Corr Coeff (After Optimizer): {np.round(np.mean(scores), 3)}\n")
    for index, target in enumerate(target_cols):
        print(f"{target} -----------------------------------------> {scores[index]}")



In [ ]:
if os.getcwd() != '/content/drive/My Drive/google_qa':
    os.chdir('./drive/My Drive/google_qa')
xlnet_tokenizer = XLNetTokenizer.from_pretrained('xlnet/tokenizer')
xlnet_model = transformer_model('xlnet')
xlnet_model.load_weights('./final_models/model-save-weights/xlnet')
c_lstm_model = tf.keras.models.load_model('final_models/c-lstm-save/c-lstm')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
file = np.load('embeddings.npz', allow_pickle = True)
vocab = file['a'].tolist()
embedding_matrix = file['b']
encoder = hub.load('final_models/universal-sentence-encoder-qa_3')
opt = pickle.load(open('optimize', 'rb'))
train_df = pd.read_csv('train.csv')
gkf = GroupKFold(n_splits = 5).split(X=train_df.url, groups = train_df.url)
for fold, (train_idx, valid_idx) in enumerate(gkf):
    if fold == 2:
        break

In [71]:
inputs = {}
inputs['question_title'] = ['Hello guys i am following an android tutorial online i wrote my code, but the app keeps crashing please help me']
inputs['question_body'] = ['''Hello guys i am following an android tutorial online i wrote my code, 
but the app keeps crashing please help me Below is the code here\'s the screenshot please bear 
with me i am a newbe, and oh am using android studio 4.0.1 i even tried writing the code from scratch but still 
got same error message over and over.''']
inputs['answer'] = ['''You should fix your erasing type parameters in some new type. Type alias is not a new type, is just an additional name for the current type.

You can do something like this:

trait Resource

class templatedResource[T](t: T) extends Resource

case class BParameter()
case class CParameter()

object B {
  case class Resource(val b: BParameter) extends templatedResource[BParameter](b)
}

object C {
  type Resource = templatedResource[CParameter]
}

def process(r: Resource) = {
  r match {
    case a: B.Resource => true
  }
}

process(B.Resource(BParameter()))
If you need to preserve the creation syntax val bResource: B.Resource = templatedResource(bParam) in order to eliminate boilerplate for end-user - you should define the function with such creation. To eliminate implementation boilerplate of the function - you can use macro or something like shapeless, I guess.''']
inputs['host'] = ['photo.stackexchange.com']
df = pd.DataFrame(inputs)
ans = function1(df)
ans.head()

Applying Preprocessing.....
Preprocessing Finished


,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,question_not_really_a_question,question_opinion_seeking,question_type_choice,question_type_compare,question_type_consequence,question_type_definition,question_type_entity,question_type_instructions,question_type_procedure,question_type_reason_explanation,question_type_spelling,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,0.881528,0.552358,0.009383,0.755049,0.657136,0.847976,0.578685,0.387877,0.015265,0.009695,0.671923,0.017431,0.003652,0.001004,0.002025,0.011693,0.768963,0.139966,0.412659,0.0,0.638099,0.959799,0.587193,0.972681,0.982002,0.91427,0.865326,0.174745,0.274657,0.906001


In [80]:
t0 = time.time()
function2(train_df.iloc[valid_idx])
print(f'Time taken : {np.round((time.time() - t0)/60, 3)} mins')

Applying Preprocessing.....
Preprocessing Finished
Spearman Corr Coeff (Before Optimizer): 0.424
Spearman Corr Coeff (After Optimizer): 0.442

question_asker_intent_understanding -----------------------------------------> 0.36167801327441984
question_body_critical -----------------------------------------> 0.6319027949072041
question_conversational -----------------------------------------> 0.532750502791162
question_expect_short_answer -----------------------------------------> 0.3316063017710274
question_fact_seeking -----------------------------------------> 0.3874655769317777
question_has_commonly_accepted_answer -----------------------------------------> 0.5358248959260236
question_interestingness_others -----------------------------------------> 0.3686050954822212
question_interestingness_self -----------------------------------------> 0.48392884201162084
question_multi_intent -----------------------------------------> 0.6243872349447608
question_not_really_a_question -----------

### References
* https://www.kaggle.com/sakami/google-quest-single-lstm?scriptVersionId=28487242 - The CNN-LSTM takes some inspiration from this kernel
* https://www.kaggle.com/christofhenkel/how-to-preprocessing-when-using-embeddings - Preprocessing of text data is inspired from this kernel
* https://huggingface.co/ - Documentation for Huggingface Library
* https://www.kaggle.com/c/google-quest-challenge/discussion/130041 - Used the hack for question_type_spelling
